# Simple Variational AutoEncoder training and inference example

**Description**: in this notebook, we showcase the training process and inference capabilities of a simple variational auto-encoder model.

## Imports, definitions and setup

The first block is needed only when the current environment doesn't have the `dlproject` package installed.
Therefore, if you already cloned the whole repository and run the `pip install -e .` command, you can skip the first block.

If you're running this notebook only on a Jupyter server, run the first block as well in order to obtain the necessary dependencies.

In [ ]:
!git clone https://github.com/peiva-git/deep_learning_project.git
%cd deep_learning_project
!pip install -e .

In [1]:
import dlproject as dlp
import tensorflow as tf

import os.path

2023-11-05 17:50:56.009846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 17:50:56.009946: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 17:50:56.012739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 17:50:56.094459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load the MNIST dataset

In [2]:
dataset_builder = dlp.data.MNISTDatasetBuilder()
dataset_builder.preprocess_dataset_simple_vae()
train_data, test_data = dataset_builder.train_x, dataset_builder.test_x

## Instantiate the model

In [3]:
vae_wrapper = dlp.models.SimpleVAE(input_dim=28 * 28, latent_dim=2)
vae_model = vae_wrapper.vae
vae_model.compile(optimizer=tf.keras.optimizers.Adam())

2023-11-05 17:51:05.994436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 17:51:05.999360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 17:51:05.999546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Train the model

Train the instantiated model on the MNIST dataset.

This block also saves a backup and a checkpoint every 20 epochs, so that you can automatically resume the training if it gets interrupted.

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist')):
    os.mkdir(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist'))
    os.mkdir(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist', 'backup'))
    os.mkdir(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist', 'model_checkpoints'))

model_dir_path = os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist')

vae_model.fit(
    train_data, train_data,
    epochs=100,
    batch_size=32,
    validation_data=(test_data, test_data),
    callbacks=[
        tf.keras.callbacks.BackupAndRestore(
            backup_dir=os.path.join(model_dir_path, 'backup'),
            save_freq=37500 # 20 * 1875, each 20 epochs
        ),
        tf.keras.callbacks.ModelCheckpoint(model_dir_path, 'model_checkpoints', save_freq=37500)
    ]
)

2023-11-05 17:51:40.976243: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-11-05 17:51:41.294045: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-11-05 17:51:41.452764: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-11-05 17:51:41.552086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/100


2023-11-05 17:51:42.760269: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-05 17:51:43.243688: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f942c0a0bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-05 17:51:43.243711: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-11-05 17:51:43.261004: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-05 17:51:43.325389: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-05 17:51:43.455826: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1868/1875 [============================>.] - ETA: 0s - loss: 188.6687

2023-11-05 17:51:49.003323: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


1875/1875 [==============================] - 8s 3ms/step - loss: 188.6075 - val_loss: 168.2643
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 166.4930 - val_loss: 163.5862
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 162.6356 - val_loss: 160.7558
Epoch 4/100
1875/1875 [==============================] - 6s 3ms/step - loss: 160.5060 - val_loss: 159.1885
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 159.0140 - val_loss: 157.9312
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 157.7761 - val_loss: 156.8272
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 156.6780 - val_loss: 155.8599
Epoch 8/100
1875/1875 [==============================] - 5s 3ms/step - loss: 155.7079 - val_loss: 155.0789
Epoch 9/100
1875/1875 [==============================] - 5s 3ms/step - loss: 154.8771 - val_loss: 154.3197
Epoch 10/100
1875/1875 [=========================

## Save the trained model

Save the just trained model for later use.

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), 'output', 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'output', 'models'))

vae_model.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_model}_mnist.keras'))
vae_wrapper.encoder.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_wrapper.encoder.name}_mnist.keras'))
vae_wrapper.decoder.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_wrapper.decoder.name}_mnist.keras'))

## Load the model

Instead of training the model, you can load it from a previously saved `.keras` file.

In [ ]:
vae_model = tf.keras.saving.load_model(os.path.join(os.getcwd(), 'output', 'models', f'{vae_model.name}_mnist.keras'))

## Visualization

Display a scatter plot of the encoded test data.